## Sets:
- J = [1:j] -> nodes of demand points(DP)
- F = [1:f] -> product families(PF)
- T = [1:t] -> planning horizont 
- I = [1:i] -> nodes of manufacturing plants(MP) 
- K = [1,k] -> nodes of storage facilities(SF) 
- st = ['Large', 'Medium', 'Small'] -> storage type/Size 

## Multidimentional Sets:
- FJ=[f,j] -> Product Families requiers by each Demand Point  
- FT=[f,st] -> Product Families able to locate in each a st facility  
- TK=[st,k] -> Potencial tipes st allowed to instal in node k 

## Parameters:
- D[j,f,t] -> annual demand of a product in an specific demand point  
- av[f,i]  -> annual availability of a MP 
- q_lo[f,st] -> Minimum amount of a PF f in a storage facility of type st -> 
- q_up[f,st] -> Maximum amount of a PF f in a a storage facility of type st -> 
- tq_lo[st] -> Global minimum of type st 
- tq_up[st] -> Global maximum of type st
### Costs and Investments
- cp[f,i] -> cost of purchase 
- ffmc[f] -> mantainance fost of a PF
- fmc[st] -> fixed cost of a SF of type 'st'
- vhc[f,st] -> operational handling cost
- Inv[st] -> Investment cost of SF type st
- FInv[f,st] -> Investment for alocating a PF 'f' in a SF of type 'st'.

## Binary Variables:
- v[k] -> The node k is adopted
- w[k,st] -> The node k is adopted with a storage capacity st
- wq[f,k,st] -> A PF 'f'f is managed in the SF 'k' of type 'st'
- q[f,k] -> A PF 'f' is managed in the SF 'k'

- x[f,i,k,t] -> Primary Links
- y[f,k,kk,t] -> Intermediate Links
- z[f,k,j,t] -> Finals Links

## Positive Variables: 
- QFC[f, k, j, t] -> Annual flow of a PD 'f' from a SF 'k' to a DP 'j'
- QFM[f, i, k, t] -> Annual purchased of 'f' from 'i' and shiped to 'k'
- QFK[f, k, kk, t] ->  Annual flow of 'f' from 'k' to 'kk'
- QTF[f, k, t] ->  Annual flow of 'f' moving across 'k'
- TQ[k,t] -> Global annual flow acorss 'k'
- QTTF[f,k,st,t]
### Cost Variables
- Capex[k] -> Total Capex of SF in 'k'
- FFInv[f,k,st] -> Product Alocation Investment
- TI -> Total Invesments
- TPC[t] -> Total Purchase Cost
- Tfmc[t] -> Total Fixed Cost
- Opex[k,t] -> Operational Cost of ST in 'k'
- TOC[t] -> Total Operational Handling Cost
- TTC1 -> Primary transport Cost
- TTC2 -> Intermediate transport Cost
- TTC3 -> Final transport Cost
- TTC -> Total Transport Cost

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyomo.environ import *
import cplex

In [2]:
df_sets = pd.read_excel('datos/inputs_supply_chain.xlsx', sheet_name="SETS")

In [3]:
T = list(df_sets.year.dropna(axis=0))
P = list(df_sets.products.dropna(axis=0))
ST = list(df_sets.facility_type.dropna(axis=0))
I = list(df_sets.nodes_i.dropna(axis=0))
K = list(df_sets.nodes_k.dropna(axis=0))
J = list(df_sets.nodes_j.dropna(axis=0))

In [4]:
dem = pd.read_excel('datos/inputs_supply_chain.xlsx', sheet_name="DEMAND")
dem.set_index(['J','P','T'], inplace=True)

In [5]:
dem.head()

demand  rand_demand
J  P  T                      
j0 p0 t0      84           45
      t1      72           81
      t2      66           54
      t3       9           41
      t4      24           83

In [6]:
df_nodes = pd.read_excel('datos/inputs_supply_chain.xlsx', sheet_name="NODES")

In [7]:
nodes_i = df_nodes[['node_i','x_i','y_i']].copy()
nodes_i.dropna(axis=0,inplace=True)
nodes_i.set_index('node_i', inplace=True)

nodes_k = df_nodes[['node_k','x_k','y_k']].copy()
nodes_k.dropna(axis=0,inplace=True)
nodes_k.set_index('node_k', inplace=True)

nodes_j = df_nodes[['node_j','x_j','y_j']].copy()
nodes_j.dropna(axis=0,inplace=True)
nodes_j.set_index('node_j', inplace=True)

In [8]:
dist_1 = {}
for i in I:
    for k in K:
        dist_1[(i,k)] = np.sqrt( (nodes_i.loc[i,'x_i']-nodes_k.loc[k,'x_k'])**2 +  (nodes_i.loc[i,'y_i']-nodes_k.loc[k,'y_k'])**2)/1000
        
dist_2 = {}
for k in K:
    for kk in K:
        dist_2[(k,kk)] = np.sqrt( (nodes_k.loc[k,'x_k']-nodes_k.loc[kk,'x_k'])**2 +  (nodes_k.loc[k,'y_k']-nodes_k.loc[kk,'y_k'])**2)/1000
        
dist_3 = {}
for k in K:
    for j in J:
        dist_3[(k,j)] = np.sqrt( (nodes_k.loc[k,'x_k']-nodes_j.loc[j,'x_j'])**2 +  (nodes_k.loc[k,'y_k']-nodes_j.loc[j,'y_j'])**2)/1000

In [9]:
df_facilities_1 = pd.read_excel('datos/inputs_supply_chain.xlsx', sheet_name="FACILITIES_1")
df_facilities_1.set_index('ST',inplace=True)

In [10]:
df_facilities_1

,low,up,capex,fixed_cost
ST,,,,
st0,0,100000,10,1.000000
st1,0,100000,12,1.095445
st2,0,100000,15,1.224745


In [11]:
df_facilities_2 = pd.read_excel('datos/inputs_supply_chain.xlsx', sheet_name="FACILITIES_2")
df_facilities_2.set_index(['ST','P'], inplace=True)

In [12]:
df_facilities_2

low    up  rand_low  rand_up  allocation_cost  allocation_rand  \
ST  P                                                                    
st0 p0    0   500        20       21         0.021442         0.011646   
    p1    0   500        11       37         0.053293         0.055509   
    p2    0   500        20       21         0.008051         0.003278   
    p3    0   500         2       34         0.067314         0.002982   
    p4    0   500        20       27         0.057761         0.065932   
    p5    0   500         3       37         0.039215         0.001745   
st1 p0    0  1000        32       43         0.016455         0.017535   
    p1    0  1000        39       61         0.024735         0.070271   
    p2    0  1000        32       85         0.016517         0.031567   
    p3    0  1000        21       88         0.002362         0.013211   
    p4    0  1000        30       85         0.066154         0.095699   
    p5    0  1000        28       66         0.057787         0.031489   
st2 p0    0  2000        45      129         0.099780         0.029288   
    p1    0  2000        47      139         0.071233         0.025074   
    p2    0  2000        53      194         0.033298         0.012069   
    p3    0  2000        50      164         0.060592         0.019553   
    p4    0  2000        43      166         0.093251         0.042051   
    p5    0  2000        53      159         0.076949         0.089900   

        handling_cost  processing_time  up_2  low_2  
ST  P                                                
st0 p0       0.000043              0.5    40      2  
    p1       0.000107              0.5    23      4  
    p2       0.000016              0.5    20      5  
    p3       0.000135              0.5    29      5  
    p4       0.000116              0.5    28      7  
    p5       0.000078              0.5    20      8  
st1 p0       0.000016              0.3    78     30  
    p1       0.000025              0.3    93     29  
    p2       0.000017              0.3    44     28  
    p3       0.000002              0.3    79     40  
    p4       0.000066              0.3    46     23  
    p5       0.000058              0.3    45     32  
st2 p0       0.000050              0.1   194     49  
    p1       0.000036              0.1   177     55  
    p2       0.000017              0.1   128     56  
    p3       0.000030              0.1   141     58  
    p4       0.000047              0.1   133     49  
    p5       0.000038              0.1   112     46

In [13]:
df_suppliers_1 = pd.read_excel('datos/inputs_supply_chain.xlsx', sheet_name="SUPPLIERS_1")
df_suppliers_1.set_index('I',inplace=True)

In [14]:
df_suppliers_1

,up
I,
i0,99999
i1,99999
i2,99999
i3,99999


In [15]:
df_suppliers_2 = pd.read_excel('datos/inputs_supply_chain.xlsx', sheet_name="SUPPLIERS_2")
df_suppliers_2.set_index(['I','P'],inplace=True)

In [16]:
df_suppliers_2

up  rand_up  purchase_cost  rand_p_cost  up_2
I  P                                                   
i0 p0  99999      168       0.002557     0.007080   295
   p1  99999      219       0.004364     0.005065   283
   p2  99999       24       0.004604     0.008270   432
   p3  99999      264       0.004101     0.001373    40
   p4  99999      458       0.008695     0.009278   455
   p5  99999       18       0.001511     0.009134   159
i1 p0  99999      305       0.002557     0.002546   452
   p1  99999       11       0.004364     0.001993   186
   p2  99999      378       0.004604     0.007313   381
   p3  99999      334       0.004101     0.000162   132
   p4  99999      412       0.008695     0.002173   255
   p5  99999      302       0.001511     0.003587    92
i2 p0  99999       55       0.002888     0.001616   447
   p1  99999      317       0.001472     0.008099   246
   p2  99999      469       0.002162     0.001548   117
   p3  99999      250       0.007729     0.000196   268
   p4  99999      297       0.007480     0.008544    56
   p5  99999      480       0.008662     0.001383   271
i3 p0  99999      280       0.008571     0.007339   127
   p1  99999      378       0.006771     0.000449    56
   p2  99999      358       0.001800     0.000307   440
   p3  99999      277       0.007088     0.008013   323
   p4  99999      197       0.005890     0.000457   492
   p5  99999      257       0.004700     0.009409   264
i4 p0  99999      280       0.005910     0.005220   131
   p1  99999       92       0.005369     0.007407    95
   p2  99999      454       0.003124     0.009516   187
   p3  99999      311       0.006080     0.005631   485
   p4  99999      243       0.005944     0.007322   412
   p5  99999      314       0.000332     0.009832   180
i5 p0  99999      142       0.007357     0.008424    41
   p1  99999       65       0.006381     0.001382   286
   p2  99999        1       0.005463     0.001501   137
   p3  99999      424       0.008716     0.007621   144
   p4  99999      268       0.008971     0.004000   249
   p5  99999      150       0.006549     0.003879   290

In [17]:
df_products = pd.read_excel('datos/inputs_supply_chain.xlsx', sheet_name="PRODUCTS")
df_products.set_index('P',inplace=True)

In [18]:
df_products

,t_primary_cost,t_intermediate_cost,t_final_cost,inventory_cost,rand_inv_cost,waiting_cost,rand_waiting_cost,mantainance_cost,rand_mant_cost
P,,,,,,,,,
p0,0.01,0.05,0.1,0.000889,0.003221,0.004133,0.002614,0.005511,0.009538
p1,0.01,0.05,0.1,0.009633,0.005720,0.005742,0.003255,0.000723,0.003783
p2,0.01,0.05,0.1,0.000127,0.002741,0.005728,0.004632,0.001833,0.001521
p3,0.01,0.05,0.1,0.008537,0.008020,0.001690,0.006865,0.000871,0.001558
p4,0.01,0.05,0.1,0.006305,0.002528,0.000504,0.008875,0.007173,0.007918
p5,0.01,0.05,0.1,0.002480,0.000638,0.004665,0.008226,0.005493,0.009036


In [19]:
def set_model():
    #SETS
    m = ConcreteModel()
    
    m.J = Set(initialize = J, doc = "Demand Points")
    m.P = Set(initialize = P, doc = "Products")
    m.T = Set(initialize = T ,doc = "Planning Horizont", ordered=True)
#     m.N = Set(initialize =  ,doc = "Nodes of Network")
    
    m.I = Set(initialize=I, doc = "Nodes of Manufacturing Plants")
    m.K = Set(initialize=K, doc = "Nodes of Storage Facilities")
    
    m.ST = Set(initialize=ST, doc = "Type of Storage Facilities")
    
    m.PJ = Set(m.P,m.J, doc = "Products requires by each demand point")
    m.PST = Set(m.P, m.ST, doc = "Products able to locate in each facility type")
    m.KST = Set(m.K, m.ST, doc = "Facility type allowed to locate in each node")

    big_m = 1000000
    
    #VARIABLES
    m.v = Var(m.K, domain=Binary, doc = "The node k is adopted", initialize=0)
    m.w = Var(m.K, m.ST, domain=Binary, doc = "The node k is adopted with a facility type st", initialize=0 )
    m.wq = Var(m.K, m.P, m.ST, domain=Binary, doc="A product 'p'f is managed in the node 'k' of type 'st'",  initialize=0)
    m.q = Var(m.K, m.P, domain=Binary ,doc="A product p is manage in the node k", initialize=0 )
    
    #Link variables
    m.x = Var(m.I,m.K,m.P,m.T, domain=Binary, doc="Primary links" , initialize=0)
    m.y = Var(m.K,m.K,m.P,m.T, domain=Binary, doc="Intermediate links", initialize=0 )
    m.z = Var(m.K,m.J,m.P,m.T, domain=Binary, doc="Final links" , initialize=0)
    
    m.QIK = Var(m.I, m.K, m.P, m.T, domain=NonNegativeReals)
    m.QKJ = Var(m.K, m.J, m.P, m.T, domain=NonNegativeReals)
    m.QKK = Var(m.K, m.K, m.P, m.T, domain=NonNegativeReals)
    
    m.QKST = Var(m.K, m.P, m.ST, m.T, domain=NonNegativeReals)
    
    m.QK = Var(m.K, m.P, m.T, domain=NonNegativeReals)
    m.QK_total = Var(m.K, m.T, domain=NonNegativeReals)
    
    m.flow_in = Var(m.K, m.P, m.T, domain=NonNegativeReals)
    m.flow_out = Var(m.K, m.P, m.T, domain=NonNegativeReals)
    
    m.capex_products = Var(m.K, m.P, m.T, domain=NonNegativeReals)
    m.capex = Var(m.K, domain=NonNegativeReals)
    m.total_capex = Var(domain=NonNegativeReals)
    m.TPC = Var(m.T, domain=NonNegativeReals)
    m.mantainance_costs = Var(m.T, domain=NonNegativeReals)
    m.administrative_costs = Var(m.T, domain=NonNegativeReals)
    m.total_opex = Var(m.T, domain=NonNegativeReals)
    m.opex = Var(m.K, m.T, domain=NonNegativeReals)
    
    m.TTC = Var(m.T, domain=NonNegativeReals)
    m.TTC_1 = Var(m.T, domain=NonNegativeReals)
    m.TTC_2 = Var(m.T, domain=NonNegativeReals)
    m.TTC_3 = Var(m.T, domain=NonNegativeReals)
    
    m.OWK = Var(m.K, m.P, m.T, m.ST, domain=NonNegativeReals)
    m.WC= Var(m.K, m.P, m.T, domain=NonNegativeReals)
    m.TWC = Var(m.T, domain=NonNegativeReals)
    
    m.npc = Var(domain=NonNegativeReals)
    
    
    ####################CONSTRAINTS####################
    
    
    m.constraints = ConstraintList()
    
    
    #Storage Facilty Allocation
    for k in m.K:
        m.constraints.add(sum(m.w[k,st] for st in m.ST) == m.v[k])
        m.constraints.add(sum(m.v[k] for k in m.K) <= 3)
        for st in m.ST:
            m.constraints.add(m.w[k,st] <= m.v[k])
    
    #Demand Supply
    for t in m.T:
        for j in m.J:
            for p in m.P:
                m.constraints.add(sum(m.QKJ[k,j,p,t] for k in m.K) >= dem.loc[(j,p,t),'demand'])
        
        
    #Mass Balance
    for k in m.K:
        for t in m.T:
            m.constraints.add(m.QK_total[k,t] == sum(m.QK[k,p,t] for p in m.P))
            
            m.constraints.add(m.QK_total[k,t] >= sum(df_facilities_1.loc[st,'low']*m.w[k,st] for st in m.ST))
            
            m.constraints.add(m.QK_total[k,t] <= sum(df_facilities_1.loc[st,'up']*m.w[k,st] for st in m.ST))
            
            
            for p in m.P:
                m.constraints.add(m.QK[k,p,t] == sum(m.QIK[i,k,p,t] for i in m.I) + sum(m.QKK[kk,k,p,t] for kk in m.K))
                
                m.constraints.add(m.flow_in[k,p,t] == sum(m.QIK[i,k,p,t] for i in m.I) + sum(m.QKK[kk,k,p,t] for kk in m.K))
                
                m.constraints.add(m.flow_out[k,p,t] == sum(m.QKK[k,kk,p,t] for kk in m.K) + sum(m.QKJ[k,j,p,t] for j in m.J))
                
                m.constraints.add(m.flow_in[k,p,t] == m.flow_out[k,p,t])
                
                m.constraints.add(m.QK[k,p,t] >= sum(df_facilities_2.loc[(st,p), 'low']*m.wq[k,p,st] for st in m.ST))
                
                m.constraints.add(m.QK[k,p,t] <= sum(df_facilities_2.loc[(st,p), 'up']*m.wq[k,p,st] for st in m.ST))

                for st in m.ST:
                    m.constraints.add( m.wq[k,p,st] <= m.v[k])
                    m.constraints.add( m.wq[k,p,st] <= m.w[k,st])
                    m.constraints.add( m.wq[k,p,st] <= m.q[k,p])
                    m.constraints.add( m.wq[k,p,st] <= m.q[k,p] + m.w[k,st] - 1)

    
    
    #Primary Links
    for i in m.I:
        for k in m.K:
            for p in m.P:
                for t in m.T:
                    m.constraints.add(m.x[i,k,p,t] <= m.v[k])
                    m.constraints.add(m.QIK[i,k,p,t] <= big_m*m.v[k])
                    m.constraints.add(m.QIK[i,k,p,t] <= big_m*m.q[k,p])
                    m.constraints.add(m.QIK[i,k,p,t] <= df_suppliers_2.loc[(i,p),'up']*m.x[i,k,p,t])
                    
    for i in m.I:
        for p in m.P:
            for t in m.T:
                m.constraints.add(sum(m.QIK[i,k,p,t] for k in m.K) <= df_suppliers_1.loc[i,'up'])
    
    #Intermediate links
    for k in m.K:
        for kk in m.K:
            for p in m.P:
                for t in m.T:
                    m.constraints.add(m.y[k,kk,p,t] <= m.v[k])
                    m.constraints.add(m.y[k,kk,p,t] <= m.v[kk])
                    m.constraints.add(m.y[k,kk,p,t] <= m.q[k,p])
                    m.constraints.add(m.y[k,kk,p,t] <= m.q[kk,p])
                    
                    m.constraints.add(m.QKK[k,kk,p,t] <= big_m*m.v[k])
                    m.constraints.add(m.QKK[k,kk,p,t] <= big_m*m.v[kk])
                    
                    m.constraints.add(m.QKK[k,kk,p,t] <= big_m*m.q[k,p])
                    m.constraints.add(m.QKK[k,kk,p,t] <= big_m*m.q[kk,p])
                    
                    m.constraints.add(m.QKK[k,kk,p,t] <= big_m*m.y[k,kk,p,t])
                    m.constraints.add(m.QKK[k,kk,p,t] <= big_m*m.y[kk,k,p,t])
                                   
    #Final Links
    for k in m.K:
        for j in m.J:
            for p in m.P:
                for t in m.T:
                    m.constraints.add(m.z[k,j,p,t] <= m.v[k])
                    m.constraints.add(m.z[k,j,p,t] <= m.q[k,p])
                    
                    m.constraints.add(m.QKJ[k,j,p,t] <= dem.loc[(j,p,t),'demand']*m.v[k])
                    m.constraints.add(m.QKJ[k,j,p,t] <= dem.loc[(j,p,t),'demand']*m.q[k,p])
                    m.constraints.add(m.QKJ[k,j,p,t] <= dem.loc[(j,p,t),'demand']*m.z[k,j,p,t])
    
    
    #Investments
    def r_capex_a(m, k, t):
        return m.capex_products[k, p, t] == df_facilities_2.loc[(st,p),'allocation_cost']*(m.q[k,p] + m.w[k,st] - 1)
    m.r_capex_a = Constraint(m.K, m.T, rule = r_capex_a)
    
    
    def r_capex_b(m, k):
        return m.capex[k] == sum(m.w[k,st]*df_facilities_1.loc[st,'capex'] for st in m.ST)
    m.r_capex_b = Constraint(m.K, rule = r_capex_b)
    
    
    def r_total_capex(m):
        return m.total_capex == sum(m.capex[k] for k in m.K)
    m.r_total_capex = Constraint(rule=r_total_capex)
    
    
    def r_purchase_cost(m, i, p, t):
        return m.TPC[t] == sum(m.QIK[i,k,p,t]*df_suppliers_2.loc[(i,p),'purchase_cost'] for i in m.I for k in m.K for p in m.P)
    m.r_purchase_cost = Constraint(m.I, m.P, m.T, rule=r_purchase_cost)
    
    
    #Operational Costs
    for t in m.T:
        m.constraints.add(m.mantainance_costs[t] == sum(m.q[k,p]*df_products.loc[p,'mantainance_cost'] for k in m.K for p in m.P))
        m.constraints.add(m.administrative_costs[t] == sum(m.w[k,st]*df_facilities_1.loc[st,'fixed_cost'] for k in m.K for p in m.P))
        m.constraints.add(m.total_opex[t] == sum(m.opex[k,t] for k in m.K))
        for k in m.K:
            m.constraints.add(m.opex[k,t] == sum(m.QKST[k,p,st,t]*df_facilities_2.loc[(st,p),'handling_cost'] for p in m.P for st in m.ST))
            for st in m.ST:
                for p in m.P:
                    m.constraints.add(m.QK[k,p,t] - df_facilities_2.loc[(st,p),'up']*(1 - m.w[k,st]) <= m.QKST[k,p,st,t] )
                    m.constraints.add(m.QKST[k,p,st,t] <= df_facilities_2.loc[(st,p),'up']*m.w[k,st])
    
    ## Transportation Costs
    for t in m.T:
        m.constraints.add( m.TTC[t] == m.TTC_1[t] + m.TTC_2[t] + m.TTC_3[t] )
        
        m.constraints.add(m.TTC_1[t] == sum(dist_1[(i,k)]*m.QIK[i,k,p,t]*df_products.loc[p,'t_primary_cost'] \
                         for i in m.I for k in m.K for p in m.P))
        
        m.constraints.add(m.TTC_2[t] == sum(dist_2[(k,kk)]*m.QKK[k,kk,p,t]*df_products.loc[ p,'t_intermediate_cost'] \
                         for k in m.K for kk in m.K for p in m.P))
        
        m.constraints.add(m.TTC_3[t] == sum(dist_3[(k,j)]*m.QKJ[k,j,p,t]*df_products.loc[p,'t_final_cost'] \
                         for k in m.K for j in m.J for p in m.P)) 
        
        
    #Inventory Costs    
#     for p in m.P:
#         m.constraints.add(m.TAS[p] == sum(order_size[(p,st),'order_size']/2 * m.wq[k,p,st] for k in m.K for st in m.ST))


    #Waiting Costs
    def r_waiting_time(m,k,p,t,st):
        return m.OWK[k,p,t,st] == sum(m.QKJ[k,j,p,t]*(df_facilities_2.loc[(st,p),'processing_time'] + dist_3[(k,j)]/30)*df_products.loc[p,'waiting_cost'] for j in m.J)
    m.r_waiting_time = Constraint(m.K, m.P, m.T, m.ST, rule=r_waiting_time)
    
    def r_waiting_cost(m,k,p,t,st):
        return m.WC[k,p,t] >= m.OWK[k,p,t,st] - big_m*(1-m.w[k,st])
    m.r_waiting_cost = Constraint(m.K, m.P, m.T, m.ST, rule=r_waiting_cost)
    
    for t in m.T:
        m.constraints.add(m.TWC[t] == sum(m.WC[k,p,t] for k in m.K for p in m.P))
        
    def r_npc(m):
        return m.npc == m.total_capex + sum(((m.TPC[t] + m.TTC[t] + m.total_opex[t] + \
                                            m.mantainance_costs[t] + \
                                            m.administrative_costs[t] + m.TWC[t])/(1+0.15)**float(t.strip()[1])) \
                                            for t in m.T)
    # + m.TSC[t]
    
    m.r_npc = Constraint(rule=r_npc)
    def OBJ(m):
        return m.npc
    m.OBJ = Objective(rule = OBJ, sense = minimize)
    
    
    return m

set_model()

In [20]:
m = set_model()
opt = SolverFactory("cplex")
results = opt.solve(m, tee=True)
results.write()


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.8.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2017.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\vitos\AppData\Local\Temp\tmpnkrdz694.cplex.log' open.
CPLEX> Problem 'C:\Users\vitos\AppData\Local\Temp\tmpm68_x3by.pyomo.lp' read.
Read time = 0.09 sec. (7.50 ticks)
CPLEX> Problem name         : C:\Users\vitos\AppData\Local\Temp\tmpm68_x3by.pyomo.lp
Objective sense      : Minimize
Variables            :   21407  [Nneg: 12487,  Binary: 8920]
Objective nonzeros   :       1
Linear constraints   :   71619  [Less: 67924,  Greater: 780,  Equal: 2915]
  Nonzeros           :  221210
  RHS nonzeros       :    3804

Variables            : Min LB: 0.0000000        Max UB: 1.000000       
Objec

In [21]:
print("Print values for all variables")
for v in m.component_data_objects(Var):
    if v.value>0:
        print( str(v), v.value ,)

Print values for all variables
v[k1] 1.0
v[k3] 1.0
v[k7] 1.0
w[k1,st1] 1.0
w[k3,st1] 1.0
w[k7,st1] 1.0
wq[k1,p0,st1] 1.0
wq[k1,p1,st1] 1.0
wq[k1,p2,st1] 1.0
wq[k1,p3,st1] 1.0
wq[k1,p4,st1] 1.0
wq[k1,p5,st1] 1.0
wq[k3,p0,st1] 1.0
wq[k3,p1,st1] 1.0
wq[k3,p2,st1] 1.0
wq[k3,p3,st1] 1.0
wq[k3,p4,st1] 1.0
wq[k3,p5,st1] 1.0
wq[k7,p0,st1] 1.0
wq[k7,p1,st1] 1.0
wq[k7,p2,st1] 1.0
wq[k7,p3,st1] 1.0
wq[k7,p4,st1] 1.0
wq[k7,p5,st1] 1.0
q[k0,p0] 1.0
q[k0,p1] 1.0
q[k0,p2] 1.0
q[k0,p3] 1.0
q[k0,p4] 1.0
q[k0,p5] 1.0
q[k1,p0] 1.0
q[k1,p1] 1.0
q[k1,p2] 1.0
q[k1,p3] 1.0
q[k1,p4] 1.0
q[k1,p5] 1.0
q[k2,p0] 1.0
q[k2,p1] 1.0
q[k2,p2] 1.0
q[k2,p3] 1.0
q[k2,p4] 1.0
q[k2,p5] 1.0
q[k3,p0] 1.0
q[k3,p1] 1.0
q[k3,p2] 1.0
q[k3,p3] 1.0
q[k3,p4] 1.0
q[k3,p5] 1.0
q[k4,p0] 1.0
q[k4,p1] 1.0
q[k4,p2] 1.0
q[k4,p3] 1.0
q[k4,p4] 1.0
q[k4,p5] 1.0
q[k5,p0] 1.0
q[k5,p1] 1.0
q[k5,p2] 1.0
q[k5,p3] 1.0
q[k5,p4] 1.0
q[k5,p5] 1.0
q[k6,p0] 1.0
q[k6,p1] 1.0
q[k6,p2] 1.0
q[k6,p3] 1.0
q[k6,p4] 1.0
q[k6,p5] 1.0
q[k7,p0] 1.0
q[k7,p1] 1.0
q

TypeError: '>' not supported between instances of 'NoneType' and 'int'